In [1]:
import os
import numpy as np
import pandas as pd
from datetime import datetime
import json
from pandas import json_normalize

In [2]:
flist= [os.path.join(path, name) for path, subdirs, files in os.walk("./files_stream/") for name in files]
nf = len(flist)
print("Number of files:", nf)
for i in range(0, nf):
    print(flist[i])

Number of files: 18
./files_stream/2022-04-15_13-32.txt
./files_stream/2022-04-15_14-50.txt
./files_stream/2022-04-15_14-56.txt
./files_stream/2022-04-15_21-34.txt
./files_stream/2022-04-15_21-49.txt
./files_stream/2022-04-15_21-50.txt
./files_stream/2022-04-15_21-51.txt
./files_stream/2022-04-15_22-00.txt
./files_stream/2022-04-15_22-09.txt
./files_stream/2022-04-15_22-11.txt
./files_stream/2022-04-15_22-14.txt
./files_stream/2022-04-15_22-27.txt
./files_stream/2022-04-15_22-31.txt
./files_stream/2022-04-15_22-46.txt
./files_stream/2022-04-15_22-47.txt
./files_stream/2022-04-15_22-48.txt
./files_stream/2022-04-15_22-57.txt
./files_stream/2022-04-15_23-46.txt


In [3]:
df= pd.DataFrame( columns= ['spect', 'game_name', 'viewer_count', 'timestamp'])
    
for file in range(0,nf):
    
    filename= flist[file]
    
    #extract timestamp
    start= './files_stream/'
    end= '.txt'
    time= filename[len(start): - len(end)]
    datetime_obj= datetime.strptime(time, '%Y-%m-%d_%H-%M')
    
    with open(filename) as f:
        data= f.readlines()
        #elimina parentesi quadra
        data= data[0]
        
        #prende in formato json
        file_j= json.loads(data)
        
        #converti json in dataframe
        for i in file_j:
            name= str(i)
            streamer= json_normalize(file_j[name])
            streamer['timestamp']= datetime_obj
            streamer['streamer']= name
            df= df.append(streamer, ignore_index= True)
        
        
        
        


In [4]:
df.shape

(55467, 5)

In [5]:
df

,spect,game_name,viewer_count,timestamp,streamer
0,"[0_0ensar0_0, 0marr_, 11ed0, 12345aar, 16lxnes...",Fortnite,2375,2022-04-15 13:32:00,Xiuder_
1,"[00rickk, 17davinc17, 1_x_2_over, 1uca74, 2001...",Just Chatting,2299,2022-04-15 13:32:00,Ivan_Grieco
2,"[00peppe_, 0zy_mandias_50, 3453373073, 34frtg4...",Just Chatting,1156,2022-04-15 13:32:00,NanniTwitch
3,"[0mar_01, 11minifish, 20th_century_boy90, 2saf...",Just Chatting,1124,2022-04-15 13:32:00,voghelita
4,"[actionshin, ago1927, aiacee, akuma_nakami, al...",Just Chatting,813,2022-04-15 13:32:00,Everyeyeit
...,...,...,...,...,...
55462,[],Call Of Duty: Modern Warfare,0,2022-04-15 23:46:00,ELPADRONDELMALE
55463,"[moobot, nightbot]",Call of Duty: Warzone,0,2022-04-15 23:46:00,GliScapokkiuni
55464,[],Call of Duty: Warzone,0,2022-04-15 23:46:00,KNMRT
55465,"[aliengathering, lurxx, sixflagsmagicmountain]",Counter-Strike: Global Offensive,0,2022-04-15 23:46:00,juzyqq


In [6]:
#df['watch only']= df['viewer_count'] - len(df['spect'])

In [7]:
df['game_name'].value_counts()

Fortnite                        5713
Call of Duty: Warzone           4532
Just Chatting                   3773
Grand Theft Auto V              2986
Call Of Duty: Modern Warfare    2831
                                ... 
Subnautica: Below Zero             1
Life is Strange                    1
Gumslinger                         1
Quattro chiacchiere                1
Mech Arena: Robot Showdown         1
Name: game_name, Length: 784, dtype: int64

In [8]:
bot_list= pd.read_csv('bot_list.csv', header= None)

In [9]:
bot_list

,0,1
0,tteyyd,66171
1,twitchdetails,65227
2,0liviajadeee,55005
3,judgejudysiayer,51936
4,thelurkertv,49506
...,...,...
5556,dreamteam5673,100
5557,chiru63,100
5558,amor_r7,100
5559,kevindeleon200,100


In [10]:
lista= bot_list[0].tolist()
lista

['tteyyd',
 'twitchdetails',
 '0liviajadeee',
 'judgejudysiayer',
 'thelurkertv',
 'nothingbutgay',
 'streamers_on_discord',
 '0_applebadapple_0',
 'agressive_sock',
 'helm',
 'raonidrugs',
 'apricotdrupefruit',
 'foregathers',
 'aliengathering',
 '0ax2',
 'community4smallstreamers',
 'socialfriends11',
 'rlvds1rirvnzu1rftq',
 'dankingaround',
 'avocadobadado',
 'kaxips06',
 'woppes',
 'raomonster',
 'bibiethumps',
 '79_69_6b_65_73',
 'troisgoy',
 'jd1d',
 'apparentlyher',
 'activeenergy',
 'randtarcedic1973',
 'srizbi',
 'bitcoinyx',
 'painfullest',
 'lorimilo',
 'anotherttvviewer',
 'slocool',
 'shiroikiseki',
 'bananennanen',
 'lurxx',
 'uzdark',
 'letsdothis_streamers',
 'vacede',
 'fixloven',
 'clearyourbrowserhistory',
 'verynaughtywife',
 'chat_fantastic',
 'stormpostor',
 'hotgirlfromoregon',
 'm0xyys_voicecrack',
 'lacyluxxx',
 'nuclearpigeons',
 'thealliancerules',
 'stormmunity',
 'streamersdiscrodserver',
 'social_growth_discord',
 'itsthefrits',
 'pboj',
 'welovemarbles',
